In [7]:
from time import time
from matplotlib import image
from matplotlib import pyplot as plt
import json
from tqdm import tqdm
import pandas as pd
import os
import json
from ipywidgets import *
from IPython.display import display, HTML
import ipywidgets as widgets
import numpy
import re

In [8]:
data = None

In [9]:
def get_game_ids(path):
    """returns a list of all game ids inside a json file"""
    global data
    with open(path) as json_file:
        data = json.load(json_file)
    game_ids = list(data.keys())
    return game_ids

In [10]:
events_l = 100
vis_df = None
events = None
default_game_ids = "2016030111"

In [11]:
all_paths=["./data/2016_regular_season.json", "./data/2016_playoffs.json"]

In [12]:
output_radio_selected = widgets.Text()
radio1 = widgets.RadioButtons(options=['Regular'])
radio2 = widgets.RadioButtons(options=['Playoff'])
radio1.index = None
radio2.index = None


game_slider = IntSlider(
    value=default_game_ids,
    min=default_game_ids,
    max=default_game_ids,
    step=1,
    description='Game:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')


def radio1_observer(sender):
    radio2.unobserve(radio2_observer, names=['value'])
    radio2.index = None
    global selected_option
    selected_option = radio1.value
    output_radio_selected.value = radio1.value
    selected_option = output_radio_selected.value
    path=all_paths[0]
    game_ids = get_game_ids(path)
    game_slider.value = min(game_ids)
    game_slider.min = min(game_ids)
    game_slider.max = max(game_ids)
    radio2.observe(radio2_observer, names=['value'])
    
    
def radio2_observer(sender):
    radio1.unobserve(radio1_observer, names=['value'])
    radio1.index = None
    global selected_option2
    output_radio_selected.value = radio2.value
    selected_option2 = output_radio_selected.value
    path=all_paths[1]
    game_ids = get_game_ids(path)
    game_slider.value = min(game_ids)
    game_slider.min = min(game_ids)
    game_slider.max = max(game_ids)
    radio1.observe(radio1_observer, names=['value'])
    
    
display(widgets.HBox([radio1,radio2]))
radio1.observe(radio1_observer, names=['value'])
radio2.observe(radio2_observer, names=['value'])


event = IntSlider(
    value=0,
    min=0,
    max=events_l-1,
    steps=1,
    description='Event: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')


display(game_slider, event)
out = Output()
display(out)


@out.capture()
def callback(game_id):
    print(game_id['new'])
    global vis_df
    vis_df = data[str(game_id['new'])]
    all_plays = pd.DataFrame(vis_df['liveData']['plays']['allPlays'])
    event_ids = pd.json_normalize(all_plays['about'])['eventId'].values
    event.min = min(event_ids)
    event.max = max(event_ids)
    
    
def plot_image(coordinates, title):
    image = plt.imread("./references/nhl_rink.png")
    fig, ax = plt.subplots(figsize=(9,4))
    ax.set_xlabel('Feet', fontweight ='bold')
    ax.set_ylabel('Feet', fontweight ='bold')
    ax.set_title(title, fontsize = 10, fontweight ='bold')
    ax.imshow(image, extent=[-100, 100, 42.5, -42.5])
    plt.plot(coordinates['0_x'],coordinates['0_y'], "ob", markersize=8)
    plt.show()
    
    
def plot_image_null():
    image = plt.imread("./references/nhl_rink.png")
    fig, ax = plt.subplots(figsize=(9,4))
    ax.set_xlabel('Feet', fontweight ='bold')
    ax.set_ylabel('Feet', fontweight ='bold')
    ax.set_title("Coordinates & Event data is not available", fontsize = 10, fontweight ='bold')
    ax.imshow(image, extent=[-100, 100, 42.5, -42.5])
    plt.show()
    
    
@out.capture()
def plot_event(e):
    out.clear_output()
    
    a = "Game Start Date: " + vis_df['gameData']['datetime']['dateTime']
    display(a)
    b = vis_df['gameData']['teams']['home']['abbreviation']+" (home) vs "+vis_df['gameData']['teams']['away']['abbreviation']+" (away)"
    display(b)
    
    df = pd.DataFrame({'Team': [vis_df['gameData']['teams']['home']['abbreviation'], vis_df['gameData']['teams']['away']['abbreviation']],
                        'Goals': [vis_df['liveData']['linescore']['teams']['home']['goals'], vis_df['liveData']['linescore']['teams']['away']['goals']],
                        'SOG': [vis_df['liveData']['linescore']['teams']['home']['shotsOnGoal'], vis_df['liveData']['linescore']['teams']['away']['shotsOnGoal']],
                        'SO Goals': [vis_df['liveData']['linescore']['teams']['home']['powerPlay'], vis_df['liveData']['linescore']['teams']['away']['powerPlay']]                    
                        })  
    display(df)
    
    current_event_id = int(e['new'])
    try:
        all_plays = pd.DataFrame(vis_df['liveData']['plays']['allPlays'])
        about = pd.json_normalize(all_plays['about'])
        specific_event_about = {"about": about.loc[about['eventId'] == current_event_id].reset_index(drop=True).to_dict()}

        event_ids = pd.json_normalize(all_plays['about'])['eventId'].values
        coordinates = all_plays.coordinates.values
        players = all_plays.players.values
        team = all_plays.team.values
        result = all_plays.result.values

        con_data = pd.DataFrame(list(zip(event_ids,coordinates,players,team,result)), columns=['event_ids','coordinates','players','team','result'])

        specific_event = con_data.loc[about['eventId'] == current_event_id].reset_index(drop=True).to_dict()
        final_dict = {**specific_event_about, **specific_event}

        current_coordinates = pd.json_normalize(specific_event["coordinates"], sep='_').to_dict(orient='records')[0]
        plot_desc = pd.json_normalize(specific_event["result"], sep='_').to_dict(orient='records')[0]
        display(plot_image(current_coordinates, plot_desc["0_description"]))
        display(final_dict)
        plt.clf()
        
    except:
        display("Coordinates & Event data is not available")
        display(plot_image_null())
        plt.clf()


game_slider.observe(callback, names=['value'])
event.observe(plot_event, names=['value'])

IntSlider(value=2016030111, continuous_update=False, description='Game:', max=2016030111, min=2016030111)

IntSlider(value=0, continuous_update=False, description='Event: ', max=99)

Output()

<Figure size 640x480 with 0 Axes>